# Intro

This notebook shows an example how to use HexTractor to transform tabular data to heterogeneous graph.

# Load libs

In [1]:
import rootutils #noqa
import autoroot #noqa

In [ ]:
import pandas as pd
import hextractor.data_sources as data_sources
import hextractor.structures as structures
import hextractor.extraction as hextract
import hextractor.visualization as viz

# Single-table data case

We will start with the simples example - where all data is in a single table. The same entity (e.g. company) can be repeated mutliple times in the table - each row represents it relation with other entities e.g. company + employee. HexTractor will handle such duplication, extracting only unique entities and relations between them.

As the data is duplicated across rows (e.g. the same company appears multiple times - as many, as employeer and other relations it has), the tool will have to de-duplicate it.

In [3]:
df = pd.DataFrame(
    [
        (1, 100, 1000, 0, 0, 25, 0, [1, 2, 3]),
        (1, 100, 1000, 1, 1, 35, 1, [1, 2]),
        (1, 100, 1000, 3, 3, 45, 0, [3, 4]),
        (2, 5000, 100000, 4, 1, 18, 1, [1, 4]),
        (2, 5000, 100000, 5, 1, 20, 1, [1, 1]),
        (2, 5000, 100000, 6, 4, 31, 0, [1, 2]),
    ],
    columns=[
        "company_id",
        "company_employees",
        "company_revenue",
        "employee_id",
        "employee_occupation",
        "employee_age",
        "employee_promotion",
        "tags",
    ],
)

df

,company_id,company_employees,company_revenue,employee_id,employee_occupation,employee_age,employee_promotion,tags
0,1,100,1000,0,0,25,0,"[1, 2, 3]"
1,1,100,1000,1,1,35,1,"[1, 2]"
2,1,100,1000,3,3,45,0,"[3, 4]"
3,2,5000,100000,4,1,18,1,"[1, 4]"
4,2,5000,100000,5,1,20,1,"[1, 1]"
5,2,5000,100000,6,4,31,0,"[1, 2]"


## Prepare graph specs

Graph specs instruct HexTractor, how to construct each entity. Specifically:

1. Node Type Param - specifies how to build a specific, unique node (e.g. company).
2. Edge Type Param - specifies how to build a specific, unique edge (e.g. company-employee relation).
3. Data Source Spec - contains mulitple Node Type Params and Edge Type Params: it tells HexTractor, what kind of entities it can find in the data and how to build them.
4. Graph Spec - contains multiple Data Source Specs: it tells HexTractor, what kind of entities it can find in the data and how to build them. From a single graph spec (With possible multiple Data Source Specs) HexTractor will build a single graph.

In [17]:
company_node_params = structures.NodeTypeParams(
    node_type_name="company",
    id_col="company_id",
    attributes=("company_employees", "company_revenue"),
    attr_type="float",
    id_as_attr=True
)

company_tags_node_params = structures.NodeTypeParams(
    node_type_name="tag",
    multivalue_source=True,
    id_col="tags",
    id_as_attr=True
)

employee_node_params = structures.NodeTypeParams(
    node_type_name="employee",
    id_col="employee_id",
    attributes=("employee_occupation", "employee_age"),
    label_col="employee_promotion",
    attr_type="long",
    id_as_attr=True
)

company_has_emp_edge_params = structures.EdgeTypeParams(
    edge_type_name="has",
    source_name="company",
    target_name="employee",
    source_id_col="company_id",
    target_id_col="employee_id",
)

company_has_tag_edge_params = structures.EdgeTypeParams(
    edge_type_name="has",
    source_name="company",
    target_name="tag",
    source_id_col="company_id",
    target_id_col="tags",
    multivalue_target=True
)

single_df_specs = data_sources.DataFrameSpecs(
    name="df1",
    node_params=(
        company_node_params,
        employee_node_params,
        company_tags_node_params,
    ),
    edge_params=(company_has_emp_edge_params, company_has_tag_edge_params),
    data_frame=df,
)

graph_specs = data_sources.GraphSpecs(data_sources=(single_df_specs,))

## Extract graph

In [23]:
hetero_g_single_table = hextract.extract_data(graph_specs)
hetero_g_single_table['tag'].x = hetero_g_single_table['tag'].x.unsqueeze(-1)

In [24]:
hetero_g_single_table

HeteroData(
  company={ x=[3, 3] },
  employee={
    x=[7, 3],
    y=[7],
  },
  tag={ x=[5, 1] },
  (company, has, employee)={ edge_index=[2, 6] },
  (company, has, tag)={ edge_index=[2, 7] }
)

In [ ]:
simple_cfg = dict(
    node_types_to_colors=dict(
        company='red',
        employee='green',
        tag='blue'
    ),
    edge_types=(("company", "has", "employee"), ("company", "has", "tag")),
    node_type_label_attr_name={"company": "x", "employee": "x", "tag": "x"},
    node_type_label_attr_idx={"company": 2, "employee": 2, "tag": 0},
    notebook_visualization=True
)


simple_viz_cfg = structures.VisualizationConfig(**simple_cfg)
simple_net_viz = viz.VisualizationBuilder.build_visualization(simple_viz_cfg, hetero_g_single_table)
simple_net_viz.show("companies.html")

companies.html


# Multi-table data case

In this case we have multiple tables, each representing different entity type. We will show how to extract graph from such data. This is how the data is usually represented in a database or a normalized data warehouse.


This will be a typical case for a relational database, where each table represents a different entity type and relations between them.

In [7]:
df_company = pd.DataFrame(
    {
        "company_id": [1, 2],
        "company_employees": [100, 5000],
        "company_revenue": [1000, 100000],
    }
)

df_employee = pd.DataFrame(
    {
        "employee_id": [0, 1, 3, 4, 5, 6],
        "employee_occupation": [0, 1, 3, 1, 1, 4],
        "employee_age": [25, 35, 45, 18, 20, 31],
        "employee_promotion": [0, 1, 0, 1, 1, 0],
    }
)

df_tags = pd.DataFrame({
    'tag': [1, 2, 3, 4]
})

df_company_2_emplopyee = pd.DataFrame(
    {
        "company_id": [1, 1, 1, 2, 2, 2],
        "employee_id": [0, 1, 3, 4, 5, 6],
    }
)

df_company_2_tag = pd.DataFrame(
    {
        "company_id": [1, 1, 1, 2, 2, 2],
        "tags": [[1, 2, 3], [1, 2], [3, 4], [1, 4], [1, 1], [1, 2]],
    }
)

In [8]:
df_company

,company_id,company_employees,company_revenue
0,1,100,1000
1,2,5000,100000


In [9]:
df_employee

,employee_id,employee_occupation,employee_age,employee_promotion
0,0,0,25,0
1,1,1,35,1
2,3,3,45,0
3,4,1,18,1
4,5,1,20,1
5,6,4,31,0


In [10]:
df_tags

,tag
0,1
1,2
2,3
3,4


In [11]:
df_company_2_emplopyee

,company_id,employee_id
0,1,0
1,1,1
2,1,3
3,2,4
4,2,5
5,2,6


In [12]:
df_company_2_tag

,company_id,tags
0,1,"[1, 2, 3]"
1,1,"[1, 2]"
2,1,"[3, 4]"
3,2,"[1, 4]"
4,2,"[1, 1]"
5,2,"[1, 2]"


## Prepare graph specs

This time - NodeTypeParams and EdgeTypeParams will be defined for each table separately. Therefore we will have multiple Data Source Specs.

In [13]:
company_node_params = structures.NodeTypeParams(
    node_type_name="company",
    id_col="company_id",
    attributes=("company_employees", "company_revenue"),
    attr_type="float",
)
company_df_source = data_sources.DataFrameSpecs(
    name="df1", node_params=(company_node_params,), data_frame=df_company
)

company_tags_node_params = structures.NodeTypeParams(
    node_type_name="tag",
    multivalue_source=False,
    id_col="tag",
    id_as_attr=True,
)
tag_df_source = data_sources.DataFrameSpecs(
    name="df2", node_params=(company_tags_node_params,), data_frame=df_tags
)

employee_node_params = structures.NodeTypeParams(
    node_type_name="employee",
    id_col="employee_id",
    attributes=("employee_occupation", "employee_age"),
    label_col="employee_promotion",
    attr_type="long",
)
employee_df_source = data_sources.DataFrameSpecs(
    name="df3", node_params=(employee_node_params,), data_frame=df_employee
)

company_has_emp_edge_params = structures.EdgeTypeParams(
    edge_type_name="has",
    source_name="company",
    target_name="employee",
    source_id_col="company_id",
    target_id_col="employee_id",
)
company_has_emp_edge_df_source = data_sources.DataFrameSpecs(
    name="df4",
    edge_params=(company_has_emp_edge_params,),
    data_frame=df_company_2_emplopyee,
)

company_has_tag_edge_params = structures.EdgeTypeParams(
    edge_type_name="has",
    source_name="company",
    target_name="tag",
    source_id_col="company_id",
    target_id_col="tags",
    multivalue_target=True,
)
company_has_tag_edge_df_source = data_sources.DataFrameSpecs(
    name="df5",
    edge_params=(company_has_tag_edge_params,),
    data_frame=df_company_2_tag,
)

graph_specs_multisource = data_sources.GraphSpecs(
    data_sources=(
        company_df_source,
        employee_df_source,
        tag_df_source,
        company_has_emp_edge_df_source,
        company_has_tag_edge_df_source,
    )
)

### Extract graph

In [14]:
hetero_g_multi = hextract.extract_data(graph_specs_multisource)

In [15]:
hetero_g_multi

HeteroData(
  company={ x=[3, 2] },
  employee={
    x=[7, 2],
    y=[7],
  },
  tag={ x=[5] },
  (company, has, employee)={ edge_index=[2, 6] },
  (company, has, tag)={ edge_index=[2, 7] }
)

# Graphs extracted in both cases should be equivalent

In [16]:
hetero_g_single_table

HeteroData(
  company={ x=[3, 2] },
  employee={
    x=[7, 2],
    y=[7],
  },
  tag={ x=[5] },
  (company, has, employee)={ edge_index=[2, 6] },
  (company, has, tag)={ edge_index=[2, 7] }
)

In [17]:
hetero_g_multi

HeteroData(
  company={ x=[3, 2] },
  employee={
    x=[7, 2],
    y=[7],
  },
  tag={ x=[5] },
  (company, has, employee)={ edge_index=[2, 6] },
  (company, has, tag)={ edge_index=[2, 7] }
)

Pay attention especially, on to how **Tag** entity was extracted from multi-values source column. We have selected an option to **squeeze single dim** - tag has only an ID, so we ensure, that is it a tensor with only one dimension. Otherwise we could have set an option **squeeze single dim** to False, which would result in a tensor with two dimensions: (N x 1). Different DL models might require different input shapes, so this option is important.